In [1]:
# From https://github.com/Stability-AI/StableCascade/blob/master/inference/text_to_image.ipynb

#!pip install pygit2==1.12.2
#from google.colab import drive
#drive.mount('/content/gdrive')
%cd /content
!git clone https://github.com/Stability-AI/StableCascade.git
#%cd /content/TabbyFooocus
#!find /content/gdrive/MyDrive/Models/loras -iname "*.safetensors" -exec cp '{}' /content/TabbyFooocus/models/loras \;
#!python entry_with_update.py --share
%cd /content/StableCascade
!pip install -r requirements.txt

/content
Cloning into 'StableCascade'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 253 (delta 69), reused 37 (delta 37), pack-reused 151
Receiving objects: 100% (253/253), 118.82 MiB | 29.81 MiB/s, done.
Resolving deltas: 100% (96/96), done.
/content/StableCascade
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-install-4sxsz6ad/warmup-scheduler_006ee999c6d14908971cf43e5bf66c20
  Running command git clone --filter=blob:none --quiet https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-install-4sxsz6ad/warmup-scheduler_006ee999c6d14908971cf43e5bf66c20
  Resolved https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to commit 6b5e8953a80aef5b324104dc0c2e9b8c34d622bd
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/pabloppp/pytorch-tools to /tmp/

In [1]:
%cd /content/StableCascade/models
!bash download_models.sh essential small-small bfloat16

import os
import yaml
import torch
from tqdm import tqdm

os.chdir('..')
from inference.utils import *
from core.utils import load_or_fail
from train import WurstCoreC, WurstCoreB

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

/content/StableCascade/models
stage_a.safetensors 100%[===================>]  70.24M   124MB/s    in 0.6s    
previewer.safetenso 100%[===================>]  15.21M  --.-KB/s    in 0.1s    
effnet_encoder.safe 100%[===================>]  77.73M   125MB/s    in 0.6s    
stage_b_lite_bf16.s 100%[===================>]   1.30G   249MB/s    in 8.3s    
stage_c_lite_bf16.s 100%[===================>]   1.92G   247MB/s    in 10s     
cuda:0


In [2]:
#Load config
# SETUP STAGE C
%cd /content/StableCascade
config_file = 'configs/inference/stage_c_3b.yaml'
with open(config_file, "r", encoding="utf-8") as file:
    loaded_config = yaml.safe_load(file)

core = WurstCoreC(config_dict=loaded_config, device=device, training=False)

# SETUP STAGE B
config_file_b = 'configs/inference/stage_b_3b.yaml'
with open(config_file_b, "r", encoding="utf-8") as file:
    config_file_b = yaml.safe_load(file)

core_b = WurstCoreB(config_dict=config_file_b, device=device, training=False)

/content/StableCascade/models
['model_version', 'effnet_checkpoint_path', 'previewer_checkpoint_path']
['model_version', 'stage_a_checkpoint_path', 'effnet_checkpoint_path']


In [3]:
# SETUP MODELS & DATA
extras = core.setup_extras_pre()
models = core.setup_models(extras)
models.generator.eval().requires_grad_(False)
print("STAGE C READY")

extras_b = core_b.setup_extras_pre()
models_b = core_b.setup_models(extras_b, skip_clip=True)
models_b = WurstCoreB.Models(**{**models_b.to_dict(), 'tokenizer': models.tokenizer, 'text_model': models.text_model})
models_b.generator.bfloat16().eval().requires_grad_(False)
print("STAGE B READY")

['transforms', 'clip_preprocess', 'gdf', 'sampling_configs', 'effnet_preprocess']


Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 127MB/s]


AttributeError: 'NoneType' object has no attribute 'items'

In [4]:
print(extras)

WurstCore.Extras(training=None, transforms=None, clip_preprocess=Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
), gdf=<gdf.GDF object at 0x7936e2bb7b20>, sampling_configs={'cfg': 5, 'sampler': <gdf.samplers.DDPMSampler object at 0x7936e2bb7940>, 'shift': 1, 'timesteps': 20}, effnet_preprocess=Compose(
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
))
